## ----------------------------------------------------------------------------------------------------------------------------

## Allow to Upload Drive into Colab

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


## ----------------------------------------------------------------------------------------------------------------------------

## Upload Google Drive Files 
#### make a folder  'drive'

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

In [5]:
%cd drive

/content/drive


## ----------------------------------------------------------------------------------------------------------------------------

## Download Tensorflow API

In [0]:
#!git clone https://github.com/tensorflow/models.git


In [24]:
%cd Tensorflow_API/models/research/

/content/drive/Tensorflow_API/models/research


In [0]:
!ls

build	 gan	    maskgan	      object_detection.egg-info  slim
cocoapi  inception  object_detection  resnet


## ----------------------------------------------------------------------------------------------------------------------------

## Install Requirements

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

## ----------------------------------------------------------------------------------------------------------------------------

## Download cocotools (if needed)

In [0]:
# !git clone https://github.com/cocodataset/cocoapi.git
# !cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

## ----------------------------------------------------------------------------------------------------------------------------

## Add Paths into PTYHONPATH

In [0]:
#%cd /content/models/research
#%mkdir train eval
%set_env PYTHONPATH=/content/drive/Tensorflow_API/models:/content/drive/Tensorflow_API/models/research:/content/drive/Tensorflow_API/models/research/slim
# export PYTHONPATH=/content/drive/Tensorflow_API/models:/content/drive/Tensorflow_API/models/research:/content/drive/Tensorflow_API/models/research/slim

## ----------------------------------------------------------------------------------------------------------------------------

## Compile proto files

In [0]:
!protoc ./object_detection/protos/*.proto --python_out=.

## ----------------------------------------------------------------------------------------------------------------------------

## Check Settings so far

In [0]:
!python /content/drive/Tensorflow_API/models/research/object_detection/builders/model_builder_test.py

In [0]:
%cd Tensorflow_API/models/research/
!ls

## ----------------------------------------------------------------------------------------------------------------------------

## TF Record Maker
### if use other datasets


In [0]:
!wget pets
!tar images
!tar annotations

In [0]:
!python object_detection/dataset_tools/create_pet_tf_record.py --label_map_path=object_detection/data/pet_label_map.pbtxt --data_dir=`pwd` --output_dir=`pwd`
!ls *.record

In [0]:
!ls

## ----------------------------------------------------------------------------------------------------------------------------

## Get Pre-trained model

In [0]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* .

In [0]:
%cd ..

## ----------------------------------------------------------------------------------------------------------------------------

## Set Configurations

In [0]:
#!cp object_detection/samples/configs/faster_rcnn_resnet101_pets.config .
!sed -i “s|PATH_TO_BE_CONFIGURED|/content/models/research|g” faster_rcnn_resnet101_pets.config
!sed -i “s|/content/models/research/pet_label_map.pbtxt|/content/models/research/object_detection/data/pet_label_map.pbtxt|g” faster_rcnn_resnet101_pets.config
!sed -i “s|/content/models/research/pet_train.record|/content/models/research/pet_train_with_masks.record|g” faster_rcnn_resnet101_pets.config
!sed -i “s|/content/models/research/pet_val.record|/content/models/research/pet_val_with_masks.record|g” faster_rcnn_resnet101_pets.config

## ----------------------------------------------------------------------------------------------------------------------------

## Training

In [0]:
!python object_detection/train.py --logtostderr --train_dir=object_detection/training --pipeline_config_path=object_detection/training/faster_rcnn_inception_v2_pets.config

In [0]:
%cd drive


## ----------------------------------------------------------------------------------------------------------------------------

## Download trained model into Drive

In [0]:
!rm -r exported_graphs; mkdir exported_graphs
!export CHECKPOINT_NUMBER=NNNNN; python object_detection/export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path faster_rcnn_resnet101_pets.config \
--trained_checkpoint_prefix train/model.ckpt-${CHECKPOINT_NUMBER} \
--output_directory exported_graphs

In [0]:
!zip -r exp_g.zip exported_graphs


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({‘title’: ‘exp_g.zip’})
uploaded.SetContentFile(‘exp_g.zip’)
uploaded.Upload()
print(‘Uploaded file with ID {}’.format(uploaded.get(‘id’)))

## ----------------------------------------------------------------------------------------------------------------------------

## Download Trained Model into Local Computer

In [0]:
gen_path = '/content/.pb'
generator.save(gen_path)
from google.colab import files
files.download(gen_path)

## ----------------------------------------------------------------------------------------------------------------------------

## Visualize

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

%%capture
!pip install import_ipynb
%cd /content
%rm -r /content/0a16ae419d9eba160ddb4f48862fb9e2
!git clone https://gist.github.com/dkatsios/0a16ae419d9eba160ddb4f48862fb9e2.git
%cd /content/0a16ae419d9eba160ddb4f48862fb9e2
import import_ipynb
from IPython.display import HTML
from AnimationDisplay import plot_results
%cd /content

In [0]:

path = '/content/images/{}.ckpt'
iterator = range(200, steps+1, 200)
HTML(plot_results(path, iterator).to_jshtml())

## ----------------------------------------------------------------------------------------------------------------------------